# Language Agent Tree Search

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/llama-index-packs/llama-index-packs-lats/examples/language_agent_tree_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[LATS (Language Agent Tree Search)](https://arxiv.org/pdf/2310.04406.pdf) by Zhou et al. combines LLM capabilities in planning, acting, and reasoning within a Monte Carlo tree search framework, allowing for deliberate and adaptive problem-solving guided by external feedback and self-reflection.

We've implemented this agent as a LlamaPack - you can either pip install it to run it out-of-the-box or call `download_llama_pack` to load the pack.

## Setup

#### Option 1. pip install llama-index-packs-lats

In [ ]:
!pip install llama-index-packs-lats

In [ ]:
# install from source
!pip install -e ../.

#### Option 2. `download_llama_pack`

In [ ]:
# from llama_index.core.llama_pack import download_llama_pack

# LATSPacks = download_llama_pack("LATSPack")

### Define Global Settings

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings


llm = OpenAI(model="gpt-3.5-turbo")
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

Settings.llm = llm
Settings.embed_model = embed_model

### Download Data

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-04-10 20:56:40--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8001::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.09s   

2024-04-10 20:56:41 (19.3 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-04-10 20:56:41--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8001::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|26

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.core.storage import StorageContext
import os


if not os.path.exists("./storage/lyft"):
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")
else:
    storage_context = StorageContext.from_defaults(persist_dir="./storage/lyft")
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir="./storage/uber")
    uber_index = load_index_from_storage(storage_context)

### Setup Tools

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [ ]:
response = uber_engine.query(
    "please provide me with the balance sheet and risk factors of Uber for the year 2021."
)
print(str(response))

The balance sheet for Uber as of December 31, 2021, shows total assets of $38,774 million, total liabilities of $23,425 million, and total equity of $15,145 million. The balance sheet includes details on various asset categories such as cash and cash equivalents, short-term investments, investments, property and equipment, intangible assets, and goodwill, among others. Liabilities include accounts payable, insurance reserves, debt, and other liabilities.

Regarding risk factors, Uber mentions concentration of credit risk related to cash, cash equivalents, investments, and receivables. The company states that these assets are potentially subject to credit risk concentration, but they have policies in place to limit exposure with any one financial institution or issuer. Uber notes that they have not experienced significant losses due to these concentrations during the periods presented. Additionally, the company highlights other receivables held by reputable insurance companies with high

## Setup Agent

Now we can setup the LATS agent.

In [ ]:
from llama_index.core.agent import AgentRunner
from llama_index.packs.lats import LATSAgentWorker


agent_worker = LATSAgentWorker.from_tools(
    query_engine_tools, llm=llm, verbose=True, num_expansions=1
)
agent = AgentRunner(agent_worker)

In [ ]:
agent_worker.num_expansions

1

## Run Some Queries

In [ ]:
# task = agent.create_task(
#     "Given the revenue growth and risk factors of Uber and Lyft, "
#     "which company is performing better? Please use concrete numbers to inform your decision."
# )

task = agent.create_task(
    "Given the risk factors of Uber and Lyft, "
    "which company is performing better? Please use concrete numbers to inform your decision."
)

In [ ]:
step_output = agent.run_step(task.task_id)

> Selecting node to expand: Observation: Given the risk factors of Uber and Lyft, which company is performing better? Please use concrete numbers to inform your decision.
> Generated new reasoning step: Thought: I need to compare the risk factors of Uber and Lyft to determine which company is performing better. I can use the financial information provided by the tools to gather concrete numbers for analysis.
Action: uber_10k
Action Input: {'input': 'risk factors'}
Observation: Some of the risk factors that could have an adverse effect on the business, financial condition, operating results, or prospects of the company include:
- Adverse effects from the COVID-19 pandemic and actions taken to mitigate it.
- Potential adverse impact if Drivers were classified differently.
- Intense competition in the mobility, delivery, and logistics industries.
- The need to lower fares or offer incentives to remain competitive.
- Historical losses and the uncertainty of achieving profitability.
- Risks

In [ ]:
step_output.task_step.step_state["root_node"].children[0].current_reasoning

[ObservationReasoningStep(observation='Given the risk factors of Uber and Lyft, which company is performing better? Please use concrete numbers to inform your decision.', return_direct=False),
 ActionReasoningStep(thought='I need to compare the risk factors of Uber and Lyft to determine which company is performing better. I can use the financial information provided by the tools to gather concrete numbers for analysis.', action='uber_10k', action_input={'input': 'risk factors'}),
 ObservationReasoningStep(observation='Some of the risk factors that could have an adverse effect on the business, financial condition, operating results, or prospects of the company include:\n- Adverse effects from the COVID-19 pandemic and actions taken to mitigate it.\n- Potential adverse impact if Drivers were classified differently.\n- Intense competition in the mobility, delivery, and logistics industries.\n- The need to lower fares or offer incentives to remain competitive.\n- Historical losses and the 

In [ ]:
step_output = agent.run_step(task.task_id)

> Selecting node to expand: Observation: Some of the risk factors that could have an adverse effect on the business, financial condition, operating results, or prospects of the company include:
- Adverse effects from the COVID-19 pandemic and actions taken to mitigate it.
- Potential adverse impact if Drivers were classified differently.
- Intense competition in the mobility, delivery, and logistics industries.
- The need to lower fares or offer incentives to remain competitive.
- Historical losses and the uncertainty of achieving profitability.
- Risks associated with attracting and maintaining a critical mass of platform users.
- Importance of maintaining and enhancing brand reputation.
- Challenges related to workplace culture and growth management.
> Generated new reasoning step: Thought: I have gathered the risk factors for Uber. Now, I need to gather similar information for Lyft to compare the risk factors of both companies.
Action: lyft_10k
Action Input: {'input': 'risk factors'

In [ ]:
step_output.task_step.step_state["root_node"].children[0].children[0].current_reasoning

[ObservationReasoningStep(observation='Given the risk factors of Uber and Lyft, which company is performing better? Please use concrete numbers to inform your decision.', return_direct=False),
 ActionReasoningStep(thought='I need to compare the risk factors of Uber and Lyft to determine which company is performing better. I can use the financial information provided by the tools to gather concrete numbers for analysis.', action='uber_10k', action_input={'input': 'risk factors'}),
 ObservationReasoningStep(observation='Some of the risk factors that could have an adverse effect on the business, financial condition, operating results, or prospects of the company include:\n- Adverse effects from the COVID-19 pandemic and actions taken to mitigate it.\n- Potential adverse impact if Drivers were classified differently.\n- Intense competition in the mobility, delivery, and logistics industries.\n- The need to lower fares or offer incentives to remain competitive.\n- Historical losses and the 

In [ ]:
step_output = agent.run_step(task.task_id)

> Selecting node to expand: Observation: Investing in the company's Class A common stock involves a high degree of risk due to various factors such as the impact of the COVID-19 pandemic, natural disasters, economic downturns, competition, operational uncertainties, the unpredictability of results, challenges in attracting and retaining drivers and riders, insurance coverage adequacy, autonomous vehicle technology, security and privacy breaches, reliance on third parties, and potential regulatory and legal challenges. Additionally, the company's operations are subject to risks related to general economic factors, operational factors, and potential geopolitical developments that could adversely affect its business, financial condition, and results of operations.
> Generated new reasoning step: Thought: I have gathered the risk factors for both Uber and Lyft. Now, I will analyze the information to compare which company is performing better based on the risk factors mentioned.
Answer: Bas

In [ ]:
step_output = agent.run_step(task.task_id)

> Selecting node to expand: Thought: I have gathered the risk factors for both Uber and Lyft. Now, I will analyze the information to compare which company is performing better based on the risk factors mentioned.
Answer: Based on the risk factors provided for both Uber and Lyft, it appears that both companies face similar challenges such as the impact of the COVID-19 pandemic, competition, operational uncertainties, and regulatory challenges. Both companies also have risks related to attracting and retaining drivers, insurance coverage, autonomous vehicle technology, security breaches, and reliance on third parties. Therefore, it seems that both Uber and Lyft are facing comparable risks in their operations.
> Generated new reasoning step: Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Based on the risk factors mentioned for Uber and Lyft, both companies face similar challenges and risks in their operations. It seems that neither compa

In [ ]:
response = agent.finalize_response(task.task_id)

In [ ]:
print(str(response))

Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Based on the risk factors mentioned for Uber and Lyft, both companies face similar challenges and risks in their operations. It seems that neither company has a significant advantage over the other in terms of risk factors.
